## **Ingesta del archivo `circuits.csv`**

In [ ]:
# Creamos un parametro para el nombre del archivo
dbutils.widgets.text("p_data_source", "")
v_data_source = dbutils.widgets.get("p_data_source")

In [ ]:
# Creamos un parametro para la fecha del archivo
dbutils.widgets.text("p_file_date", "")
v_file_date = dbutils.widgets.get("p_file_date")

In [ ]:
# Llamamos al notebook que contiene las variables de configuración
%run "../utils/configuration"

In [ ]:
# Llamamos al notebook que contiene funciones comunes
%run "../utils/common_functions"

### Paso 1 - Crear tabla **f1_raw.circuits**

In [ ]:
%sql
DROP TABLE IF EXISTS f1_raw.circuits;
CREATE EXTERNAL TABLE IF NOT EXISTS f1_raw.circuits(
circuitId INT,
circuitRef STRING,
name STRING,
location STRING,
country STRING,
lat DOUBLE,
lng DOUBLE,
alt INT,
url STRING
)
USING CSV
OPTIONS (path "/mnt/formula1dl/raw/circuits.csv", header true)
-- Esta otra forma duplica la primera fila, para usarla como header y como primera fila
-- TBLPROPERTIES("skip.header.line.count"="1")
-- LOCATION "/mnt/formula1dl/raw/circuits.csv";

### Paso 2 - Leer el archivo CSV

In [ ]:
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, DoubleType

In [ ]:
circuits_schema = StructType(fields=[StructField("circuitId", IntegerType(), False),
                                     StructField("circuitRef", StringType(), True),
                                     StructField("name", StringType(), True),
                                     StructField("location", StringType(), True),
                                     StructField("country", StringType(), True),
                                     StructField("lat", DoubleType(), True),
                                     StructField("lng", DoubleType(), True),
                                     StructField("alt", IntegerType(), True),
                                     StructField("url", StringType(), True)
])

In [ ]:
# El parámetro "raw_folder_path" se encuentra en el notebook "configuration"
# El parámetro "v_file_date" se encuentra en el notebook e indicamos su valor en tiempo de ejecución
circuits_df = spark.read \
.option("header", True) \
.schema(circuits_schema) \
.csv(f"{raw_folder_path}/circuits.csv")
#.csv(f"{raw_folder_path}/{v_file_date}/circuits.csv")

### Paso 3 - Seleccionar sólo las columnas necesarias

In [ ]:
from pyspark.sql.functions import col

In [ ]:
circuits_selected_df = circuits_df.select(col("circuitId"), col("circuitRef"), col("name"), col("location"), col("country"), col("lat"), col("lng"), col("alt"))

### Paso 4 - Cambiar el nombre de las columnas

In [ ]:
from pyspark.sql.functions import lit

In [ ]:
circuits_renamed_df = circuits_selected_df.withColumnRenamed("circuitId", "circuit_id") \
.withColumnRenamed("circuitRef", "circuit_ref") \
.withColumnRenamed("lat", "latitude") \
.withColumnRenamed("lng", "longitude") \
.withColumnRenamed("alt", "altitude") \
.withColumn("data_source", lit(v_data_source)) \
.withColumn("file_date", lit(v_file_date))

### Paso 5 - Añadir la fecha de ingestión al dataframe

In [ ]:
# La función "add_ingestion_date()" se encuentra en el notebook "common_functions"
# La fecha de ingestión será del tipo timestamp
circuits_final_df = add_ingestion_date(circuits_renamed_df)

### Paso 6 - Escribir datos en el datalake como parquet y crear la tabla **circuits** en la base de datos **f1_processed**

In [ ]:
# Recordar que Databricks utiliza Hive para la gestión de tablas
# Esto quiere decir que una tabla en Hive se asocia con un directorio específico que contiene archivos de datos
# Aunque los datos subyacentes son solo archivos en un sistema de archivos, Hive proporciona una interfaz que presenta 
# estos datos como tablas.
# La base de datos tambien es un directorio. Podemos crearla manualmente, como con lenguaje SQL.
# Hive permite a los usuarios interactuar con esos datos utilizando un lenguaje similar a SQL
# En resumen, solo se esta creando un directorio y un archivo
# Escribimos el archivo con formato PARQUET en la base de datos "f1_processed" y en la tabla "circuits"
circuits_final_df.write.mode("overwrite").format("parquet").saveAsTable("f1_processed.circuits")

In [ ]:
dbutils.notebook.exit("Success")